In [1]:
import json
import os
import pandas as pd
import time

from dotenv import load_dotenv
from openrouteservice import client, directions, isochrones, places

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [18]:
output_dir = '../data/curated/'

## set variables
load_dotenv()
tokens = []
for i in range(1,24): # number of keys
    token = os.environ.get(f'token_{i}')
    tokens.append(token)
category_id = [604] # railway station
ptv_dict = {}

In [19]:
## read apartment data
property_data = json.load(open( f"{output_dir}properties_processed.json" ))
print(len(property_data['Coordinates']))

13227


In [17]:
token = tokens[20]

## set x and y
x = len(ptv_dict)
y = len(ptv_dict)+500 ## from smallest API quota
if y > len(property_data['Coordinates'].keys()):
    y = len(property_data['Coordinates'].keys())

for index in range(x,y):
    backwards = list(map(float,property_data['Coordinates'][str(index)][1:-1].split(',')))
    coords = [backwards[1], backwards[0]]
    ors = client.Client(key=token)

    ## find search region
    params_iso = {'locations': [coords],
                'profile': 'driving-car',
                'range':[1800] # 30 min drive
                }
    iso = ors.isochrones(**params_iso)

    ## find poi
    query = {'request': 'pois',
            'geojson': iso['features'][0]['geometry'],
            'filter_category_ids': category_id,
            'sortby':'distance',
            'limit': 2}
    features = ors.places(**query)['features']
    destination = features[0]['geometry']['coordinates']
    all = [(str(coords[0]),str(coords[1])),(str(destination[0]),str(destination[1]))]

    ## get car route
    request = {'coordinates': all,
            'profile': 'driving-car',
            'geometry': 'true',
            'format_out':'geojson'}
    route = ors.directions(**request)
    info = route['features'][0]['properties']['summary']

    ## add information
    ptv_dict[str(index)] = info
    time.sleep(0.5)

ApiError: 403 ({'error': 'Access to this API has been disallowed'})

In [ ]:
print(len(ptv_dict))

In [231]:
property_data['PTV'] = ptv_dict

## write json
json.dump(property_data, open(f"{output_dir}closest_train.json", 'w'))

# write csv
data = pd.read_json(f"{output_dir}closest_train.json")
data.to_csv(f"{output_dir}closest_train.csv")